<a href="https://colab.research.google.com/github/cobang0111/DRL_2023_Fall/blob/main/pole_SAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade pybullet==3.1.6 imageio-ffmpeg stable-baselines3

import os
import time
import math

import numpy as np
import imageio_ffmpeg
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from base64 import b64encode
from IPython.display import HTML
import gym
from gym import spaces

import pybullet
import pybullet_data
#from stable_baselines3.sac.policies import MlpPolicy
#from stable_baselines3 import SAC



In [3]:
!pip install 'shimmy>=0.2.1'

In [14]:
robot = pybullet.loadURDF("/content/Assem_model1/urdf/Assem_model1.urdf",[0,0,0],useFixedBase=1)
joint_info = pybullet.getJointState(robot, 0)
print(joint_info)

(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [62]:
from stable_baselines3.sac.policies import MlpPolicy
from stable_baselines3 import SAC


# Simulation & Environment
pybullet.connect(pybullet.DIRECT) #Open pybulle simulation
pybullet.resetSimulation()
#pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
#print(pybullet_data.getDataPath())
pybullet.setGravity(0, 0, -9.81) # define x,y,z gravity constants
pybullet.setTimeStep(0.01) #Time step
robot = pybullet.loadURDF("/content/Assem_model1/urdf/Assem_model1.urdf",[0,0,0],useFixedBase=1)

# camera parameters
cam_target_pos = [0, -0.2, 0.2]
cam_distance = 1.05
cam_yaw, cam_pitch, cam_roll = -50, -40, 0
cam_width, cam_height = 480, 360
cam_up, cam_up_axis_idx, cam_near_plane, cam_far_plane, cam_fov = [0, 0, 1], 2, 0.01, 100, 60


class PoleEnv(gym.Env):
    def __init__(self):
        super(PoleEnv, self).__init__()
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(2,))
        self.robot = pybullet.loadURDF("/content/Assem_model1/urdf/Assem_model1.urdf",[0,0,0],useFixedBase=1)

    def step(self, action):
        # Apply torque to the joint
        velocity = action[0]
        pybullet.setJointMotorControl2(self.robot, 0, pybullet.VELOCITY_CONTROL, targetVelocity=velocity)

        # Step the simulation
        pybullet.stepSimulation()

        # Get the joint's angle and velocity
        joint_info = pybullet.getJointState(self.robot, 0)
        joint_angle = joint_info[0]
        joint_velocity = joint_info[1]

        # Calculate reward
        reward = 1 if np.radians(-90) < joint_angle < np.radians(-70) else 0  # Reward is 1 if the pole is upright

        # The episode is done if the pole is more than some degrees from vertical
        done = np.radians(-70) < joint_angle or np.radians(-90) > joint_angle

        return np.array([joint_angle, joint_velocity]), reward, done, {}

    def reset(self):
        pybullet.resetSimulation()
        self.robot = pybullet.loadURDF("/content/Assem_model1/urdf/Assem_model1.urdf",[0,0,0],useFixedBase=1)

        initial_angle = np.random.uniform(np.radians(-90), np.radians(-70))  # Choose a random initial angle within +-12 degrees
        pybullet.resetJointState(self.robot, 0, initial_angle)  # Set the initial angle of the joint
        joint_info = pybullet.getJointState(self.robot, 0)  # Get the initial joint state
        joint_angle = joint_info[0]
        joint_velocity = joint_info[1]
        return np.array([joint_angle, joint_velocity])

    def render(self):
        cam_view_matrix = pybullet.computeViewMatrixFromYawPitchRoll(cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx)
        cam_projection_matrix = pybullet.computeProjectionMatrixFOV(cam_fov, cam_width*1./cam_height, cam_near_plane, cam_far_plane)
        image = pybullet.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]
        image = np.ascontiguousarray(image)
        return image

    def close(self):
        pybullet.disconnect()

# Create the environment
env = PoleEnv()

model = SAC('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=1000)
model.save("sac_pole_env")

del model # remove to demonstrate saving and loading

model = SAC.load("sac_pole_env")

vid = imageio_ffmpeg.write_frames('vid.mp4', (cam_width, cam_height), fps=30)
vid.send(None) # seed the video writer with a blank frame

# Enjoy trained agent
for i in range(1):
    obs = env.reset()
    for j in range(500):
        action, _states = model.predict(obs)
        obs, rewards, done, info = env.step(action)
        image = env.render()
        vid.send(image)
        if done:
            break

vid.close()
env.close()


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [64]:
# Play recorded video

mp4 = open('vid.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)